In [1]:
# import the package 
import cobra

import pandas as pd
import numpy as np

pd.options.display.max_rows = 4000

from matplotlib import pyplot as plt
plt.style.use('ggplot')

In [102]:
ipfa = cobra.io.read_sbml_model("iPFA/ipfasbml.xml")

Model does not contain SBML fbc package information.
'' is not a valid SBML 'SId'.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00001_c "H2O">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00002_c "ATP">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00003_c "NAD+">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00004_c "NADH">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00005_c "NADPH">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00006_c "NADP+">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Speci

In [2]:
ipfal21 = cobra.io.load_json_model("FBA_Carey_models/iPfal21.json")

In [ ]:
print(ipfa.compartments)
print(ipfal21.compartments)

In [ ]:
# find all substrates of the reaction
biomets = ipfa.reactions.Biomass_rxn_c.metabolites #r_4041 = biomassreaction 8

s = list(biomets.values()) #stoichiometry of the metabolites in the reaction
m = list(biomets.keys()) # metabolite in the reaction

# print a table of metabolites and their stoichiometry

metname = [mi.name for mi in m]# names of the metabolite in the reaction

# table printing (this is very messy)

ipfa_biomass = pd.DataFrame.from_records(map(list, zip(*[m,metname,s])),columns=['ID', 'metabolite','stoichiometry'])
print(len(ipfa_biomass))
ipfa_biomass.sort_values(by='stoichiometry', ascending=False)


In [ ]:
# find all substrates of the biomass reaction
biomets = ipfal21.reactions.biomass.metabolites 

s = list(biomets.values()) #stoichiometry of the metabolites in the reaction
m = list(biomets.keys()) # ID of the metabolite in the reaction

# print a table of metabolites and their stoichiometry

metname = [mi.name for mi in m]# names of the metabolite in the reaction

# table printing (this is very messy)

biomass_ipfal = pd.DataFrame.from_records(map(list, zip(*[m,metname,s])),columns=['ID', 'metabolite','stoichiometry'])
print(len(biomass_ipfal))
biomass_ipfal.sort_values(by='stoichiometry', ascending=False)

### Find metabolites not in both models 

In [ ]:
#biomass_ipfal.metabolite[biomass_ipfal.metabolite.isin(biomass.metabolite)]
kegg_ids = []
ids = []
for i,met in enumerate(m):
    try:
        kegg = met.annotation['kegg.compound']
    except:
        kegg=''
    kegg_ids.extend(kegg)
    ids.extend([i for _ in range(len(kegg))])

def return_kegg_id(metabolite):
    try:
        kegg = metabolite.annotation['kegg.compound']
    except:
        kegg=''
    return kegg


biomass_ipfal['KEGG'] = biomass_ipfal['ID'].apply(return_kegg_id)

#### iPFA biomass exclusive metabolites

In [ ]:
ipfa_kegg = [m.id[:-2] for m in ipfa_biomass.ID]
ipfa_biomass['KEGG']=ipfa_kegg
print(len(ipfa_biomass))
print(len(biomass_ipfal))
print(len(ipfa_biomass[ipfa_biomass.KEGG.isin(kegg_ids)]))
# Venn part of included in both biomass functions
ipfa_venn = ipfa_biomass[ipfa_biomass.KEGG.isin(kegg_ids)].sort_values(by='metabolite', ascending='True')
ipfa_venn

In [ ]:
not_venn_ipfa_biomass=ipfa_biomass[~ipfa_biomass.KEGG.isin(kegg_ids)].sort_values(by='metabolite', ascending='True')
not_venn_ipfa_biomass

#### ipfal biomass of in both included 

In [ ]:
inter, ids_l, ids_i = np.intersect1d(np.asarray(kegg_ids), np.asarray(ipfa_kegg), return_indices=True)
not_included = np.asarray(ids)[~np.isin(np.arange(len(ids)), ids_l)]
venn_ipfal = biomass_ipfal.iloc[np.asarray(ids)[ids_l]].sort_values(by='metabolite', ascending='True')
venn_ipfal

In [ ]:
0.134900*(0.45* 1/175.11 *1000)**-1

In [ ]:
biomass_ipfal[~biomass_ipfal.index.isin(venn_ipfal.index)].sort_values(by='metabolite', ascending='True')

In [ ]:
not_ipfa = biomass_ipfal[~biomass_ipfal.index.isin(venn_ipfal.index)].sort_values(by='metabolite', ascending='True')

to_add=['Fe2','Fe3','L_Malate','Riboflavin','lac_L[e]','hemozoin_e','2_Octaprenyl_6_hydroxyphenol','2_Oxoglutarate','Oxidized_glutathione']
not_ipfa = not_ipfa[not_ipfa['metabolite'].isin(to_add)]
not_ipfa = not_ipfa.drop(columns=['KEGG'])
not_ipfa.columns = ['ID_y','metabolite_x','stoichiometry_x']
not_ipfa

In [ ]:
0.01/(1.63/70)

### Rewrite iPFA biomass df for ipfal

In [ ]:
# Get correct order of dfs
venn_ipfal = biomass_ipfal.iloc[np.asarray(ids)[ids_l]].copy()
ipfa_venn = ipfa_biomass[ipfa_biomass.KEGG.isin(kegg_ids)].sort_values(by='KEGG', ascending='True')
# Give ipfal single same KEGG ID as ipfa
ipfa_ID = sorted(ipfa_venn.KEGG)
venn_ipfal['ipfa_ID'] = ipfa_ID
# Rename ipfal columns, prep for merge
venn_ipfal.columns = ['ID', 'metabolite', 'stoichiometry', 'KEGG_IDS', 'KEGG']
del ipfa_ID
# merge df on single string KEGG IDs
df_merge = pd.merge(ipfa_venn, venn_ipfal, on='KEGG')
# drop not relevant columns
df_merge = df_merge.drop(columns=['ID_x','metabolite_y','stoichiometry_y','KEGG_IDS','KEGG'])
df_merge.head()

#### iPFA biomass not included metabolites analysis
Exclude lipids as those will be added with SLIMEr and find matching metabolites for iPFA biomass metabolites, which could exist in the model just not in biomass

In [ ]:
# Lipids to exclude
lipids = ['Cholesterol','Phosphatidylcholine','Phosphatidylserine','Phosphatidylethanolamine',
          'Sphingomyelin','Triacylglycerol','Cardiolipin','1,2-Diacyl-sn-glycerol',
          'Octadecanoic acid','Hexadecanoic acid', '(9Z)-Octadecenoic acid', 'Tetradecanoic acid', 'Linoleate']
# Metabolites found with their ipfal IDs
meta_nme = {'D-myo-Inositol 1,4,5-trisphosphate':'mi145p_c',
            'FMN':'fmn_c',
            'GDP-L-fucose':'gdpfuc_c',
            'GDP-mannose':'gdpmann_c',
            'Geranylgeranyl diphosphate':'ggdp_c',
            'Isopentenyl diphosphate':'ipdp_c',
            'UDP-N-acetyl-D-glucosamine':'uacgam_c',
            'UDP-glucose':'udpg_c',
            'Ubiquinone-8':'q8_c'}
# Totally missing as metabolite from ipfal
ipfal21_missing ={'5-Methyltetrahydrofolate':'C00440_c', # uses 5_10_Methylenetetrahydrofolate to directly produce Tetrahydrofolate not the intermediate step of 5-Methyltetrahydrofolate
                  'Biotin':'C00120_c', # KEGG has part of pathway documented (fatty acid biosynthesis) but not the enzymes close to biotin
                  # biotin not so important  doi: 10.1073/pnas.1800717115
                  'Methylcobalamin':'C06453_c', # Vitamine B12 maybe add https://doi.org/10.1016/j.jinorgbio.2007.01.006
                  'Protein N6-(lipoyl)lysine':'C16237_c', # could be included possibly pyruvate dehydrogenase, a-ketoglutarate dehydrogenase, grouped reaction in ipfal
                  #doi: 10.1111/j.1365-2958.2007.05592.x 
                  'UDP-D-galactose':'C00052_c' # bind for glyco-lipids/proteins should be added https://doi.org/10.1186/s12936-015-0949-z
               }
not_venn_ipfa_biomass_no_lipids = not_venn_ipfa_biomass[~not_venn_ipfa_biomass.metabolite.isin(lipids)]
not_venn_ipfa_biomass_no_lipids[~not_venn_ipfa_biomass_no_lipids.metabolite.isin(meta_nme.keys())]

In [ ]:
ipfal_found = not_venn_ipfa_biomass_no_lipids[not_venn_ipfa_biomass_no_lipids.metabolite.isin(meta_nme.keys())].copy()
ipfal_found['ID_y']=meta_nme.values()
ipfal_found = ipfal_found.drop(columns=['ID','KEGG'])
ipfal_found.columns= ['metabolite_x','stoichiometry_x','ID_y']
biomass_new = pd.concat([df_merge,ipfal_found])

In [ ]:
biomass_new.head()

#### Final biomass no lipids

In [ ]:
not_ipfa = biomass_ipfal[~biomass_ipfal.index.isin(venn_ipfal.index)].sort_values(by='metabolite', ascending='True')

to_add=['Fe2','Fe3','L_Malate','Riboflavin','lac_L[e]','hemozoin_e','2_Octaprenyl_6_hydroxyphenol','2_Oxoglutarate','Oxidized_glutathione']
not_ipfa = not_ipfa[not_ipfa['metabolite'].isin(to_add)]
not_ipfa = not_ipfa.drop(columns=['KEGG'])
not_ipfa.columns = ['ID_y','metabolite_x','stoichiometry_x']
total_biomass = pd.concat([not_ipfa,biomass_new])
total_biomass = total_biomass.drop(columns='metabolite_x')

#### Biomass reaction writer

In [ ]:
mini_addition = dict(zip(ipfal_found['ID_y'],ipfal_found['stoichiometry_x']))
# q8_c dead end metabolite have to go thro reactions 

In [ ]:
def mke_str(item):
    try:
        return str(item.id)
    except:
        return item
biomass_ipfal['ID']= biomass_ipfal['ID'].apply(mke_str)  
meta_test = dict(zip(biomass_ipfal['ID'], biomass_ipfal['stoichiometry']))
meta_test.update(mini_addition)

In [ ]:
def mke_str(item):
    try:
        return str(item.id)
    except:
        return item
total_biomass['ID_y'] = total_biomass['ID_y'].apply(mke_str)
meta_dict = dict(zip(total_biomass['ID_y'], total_biomass['stoichiometry_x']))
meta_dict['biomass_c']=1.0

In [ ]:

with ipfal21:
    # defining new biomass reaction
    new_biomass_rec = cobra.Reaction('new_ipfa')
    new_biomass_rec.name = 'reaction follows iPFA 2017'
    ipfal21.add_reactions([new_biomass_rec])
    # Q8_ as of now dead end metabolite
    #meta_dict.pop('q8_c')
    # Dict to give Reaction metabolites and corresponding stoich factors
    ipfal21.reactions.new_ipfa.add_metabolites(meta_dict)
    
    ipfal21.objective = 'new_ipfa'
    ipfal21.reactions.biomass.knock_out()
    
    print(ipfal21.objective.expression)
    print(ipfal21.optimize())

### Understanding of metabolites and reactions

In [ ]:
ipfal21.metabolites.fe2_c #https://www.sciencedirect.com/science/article/pii/S0166685199000535

In [ ]:
ipfal21.metabolites.chsterol_c

In [ ]:
ipfal21.reactions.CHSTEROLt.annotation

#### iPFA stoichiometric value understanding

In [ ]:
ipfa.metabolites.C00157_r

In [ ]:
'''
As example the phospholipid PC:C00157_r1)))

In [ ]:
ipfal21.optimize()

In [ ]:
ipfal21.metabolites.alatrna_c.summary()

In [ ]:
ipfa.metabolites.get_by_id('C16240_c')

In [ ]:
ipfa.reactions.R07771_m

In [ ]:
ipfa.metabolites.C00725_c

In [ ]:
ipfa.reactions.R07770_m

In [ ]:
# Galactose only produced for biomass could it be work around for lactose?
ipfa.metabolites.get_by_id('C00052_c')

In [ ]:
ipfa.reactions.R00291_c

### Exchange reactions

In [ ]:
pepe_ex = ipfa.exchanges
df_ex_pepe = pd.DataFrame(pepe_ex, columns=['Reaction'])
df_ex_pepe['Metabolites'] = [list(df_ex_pepe.iloc[x][0].metabolites.keys())[0] for x in range(len(df_ex_pepe))]
df_ex_pepe['Bounds'] = [df_ex_pepe.iloc[x][0].bounds for x in range(len(df_ex_pepe))]

In [ ]:
carey_ex = ipfal21.exchanges
df_ex_carey = pd.DataFrame(carey_ex, columns=['Reaction'])
df_ex_carey['Metabolites'] = [list(df_ex_carey.iloc[x][0].metabolites.keys())[0] for x in range(len(df_ex_carey))]
df_ex_carey['Bounds'] = [df_ex_carey.iloc[x][0].bounds for x in range(len(df_ex_carey))]


In [ ]:
pepe_ex_mets = [list(pepe_ex[x].metabolites.keys())[0].id[:-2] for x in range(len(pepe_ex))]
pepe_ex_ids = [list(pepe_ex[x].metabolites.keys())[0].id for x in range(len(pepe_ex))]

ex_kegg_ids = []
ex_ids = []
ex_df = []
for i,rec in enumerate(carey_ex):
    try:
        met = list(rec.metabolites.keys())[0]
        kegg = met.annotation['kegg.compound']
    except:
        kegg=''
    ex_kegg_ids.extend(kegg)
    ex_ids.extend([i for _ in range(len(kegg))])
    ex_df.append(kegg)
#carey_ex_mets = [.name for x in range(len(carey_ex))]

In [ ]:
inter, ids_l, ids_i = np.intersect1d(np.asarray(ex_kegg_ids), np.asarray(pepe_ex_mets), return_indices=True)
inter

In [ ]:
df_ex_pepe['Name']=[df_ex_pepe.Metabolites.iloc[x].name for x in range(len(df_ex_pepe))]
df_ex_pepe.iloc[np.isin(np.arange(len(pepe_ex_ids)), ids_i)]

In [ ]:
not_included = np.asarray(ex_ids)[~np.isin(np.arange(len(ex_ids)), ids_l)]
included = np.asarray(ex_ids)[np.isin(np.arange(len(ex_ids)), ids_l)]

df_ex_carey['KEGG']=ex_df
print(len(included))
print(len(not_included))
print(len(df_ex_carey))
df_ex_carey#.iloc[included]

In [ ]:
ipbe = cobra.io.load_matlab_model('/home/karnet/Downloads/ipbeblood.mat')

In [ ]:
ipbe.metabolites.C00157_r

In [101]:
ipbe.reactions.R01321_r	

NameError: name 'ipbe' is not defined

# Functions to create SLIME reactions

In [3]:
def SLIMEr_react_dict(metabolite:str) -> dict:
    '''
    Function to produce metabolite dict for SLIME reaction of lipid species 'metabolite'
    RETURN
        meta_dict: keys = metabolite name strings, values = stoichmetric factor
    '''
    edukt = metabolite
    to_slice_start = metabolite.find('(')
    to_slice_end = metabolite.find(')')
    #o_lipid  = metabolite.find('O')
    if to_slice_start!=-1:
        metabolite = metabolite[:to_slice_start-1] + metabolite[to_slice_end+1:]
    metabolite = metabolite.replace('_', ' ')
    products = metabolite.split(' ')
    
    if 'SM' in metabolite or 'Cer' in metabolite:
        products = products + ['18:1']

    meta_dict = {product:1 for product in products}
    meta_dict[edukt]=-1

    return meta_dict
def create_species_dict():
    return

def create_species(species_info):
    print(species_info)
    if type(species_info)!=dict:
        print(species_info)
    new_meta = cobra.Metabolite(species_info_dict)
    print(new_meta)    
    return new_meta

def check_meta_exists(model, meta_id):
    return meta_id in model.metabolites

def create_reaction(model, react_name:list, meta_dict:dict):
    for meta in meta_dict.keys():
        if not check_meta_exists(model, meta):
            print('No existing metabolite '+meta)
            return model
        
    # defining new reaction
    reaction = cobra.Reaction(react_name[0])
    reaction.name = 'SLIME reaction' + react_name[1]
    reaction.subsystem = 'SLIMEr reaction, Lipid metabolism'
    reaction.lower_bound = 0.  # This is the default
    reaction.upper_bound = 1000.  # This is the default 
    model.add_reactions([reaction])

    model.reactions.get_by_id(reaction.id).add_metabolites(meta_dict)
    print(model.reactions.get_by_id(reaction.id))

    return model


In [166]:
ipfal21 = cobra.io.load_json_model("FBA_Carey_models/iPfal21.json")

In [4]:
test = {'CE': 1, '16:0': 1, 'CE 16:0': -1}
rct_lst_str = ['SLIME_xolest160','CE 16:0']
create_reaction(ipfal21,rct_lst_str,test)

No existing metabolite CE


Name,iPfal19_v1
Memory address,7fe28e4811e0
Number of metabolites,983
Number of reactions,1233
Number of genes,475
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"c, e, ap, m, fv"


In [5]:
ipfal21.compartments

{'cytoplasm': 'c',
 'extracellular': 'e',
 'apicoplast': 'ap',
 'mitochondrion': 'm',
 'food_vacuole': 'fv'}

In [6]:
species_name_dict={'CE': ['xolest_bkb','cholesterol-ester'],
                    'PC':['pc_bkb','Phosphatidylcholine'],
                    'PE':['pe_bkb','Phosphatidylethanolamine'],
                    'PS':['ps_bkb','Phosphatidylserine'],
                    'PG':['pg_bkb','Phosphatidylglycerol'],
                    'SM':['sphmyln_bkb','Sphingomyelin'],
                    'FreeChol':['chsterol_bkb','Cholesterol'],
                    'TAG':['tag_bkb','Triacylglycerol'],
                    'DAG':['12dgr_bkb','1,2-Diacyl-sn-glycerol'],
                    'Cer':['cer_bkb','Ceramide'],
                    'DHSM':['dhsphmyln_bkb','Dihydrosphingomyelin'],
                    'pseudo head':['pseudo_h_bkb','Pseudo head lipid']}

for species in species_name_dict:
    sid = species_name_dict[species][0]
    sname = species_name_dict[species][1]+' backbone' 
    new_meta = cobra.Metabolite(id=sid,name=sname, compartment='cytoplasm')
    ipfal21.add_metabolites([new_meta])

In [7]:
metas_df = pd.read_csv('Datasets/slimeR_pseudo_all_percent.tsv',sep='\t',index_col=0)
metas_df = metas_df.drop('LPC')

In [8]:
meta_ids = []
for meta in metas_df.index:
    if meta[0].isdigit():
        sid = 'c'+meta.replace(':','')+'_chain'
        sname = 'Side chain '+meta 
        new_meta = cobra.Metabolite(id=sid,name=sname, compartment='cytoplasm')
        ipfal21.add_metabolites([new_meta])
    else:
        sid = species_name_dict[meta][0]
        sname = species_name_dict[meta][1]+' backbone' 
        new_meta = cobra.Metabolite(id=sid,name=sname, compartment='cytoplasm')
        ipfal21.add_metabolites([new_meta])
    meta_ids.append(sid)

In [9]:
metas_df['ids'] = meta_ids

In [13]:
def create_reaction(model, react_name:list, meta_dict:dict):
    if react_name[0] in model.reactions:
        print('Reaction already exists, delete it first '+ react_name[0])
        return model
    
    for meta in meta_dict.keys():
        if not check_meta_exists(model, meta):
            print('No existing metabolite ' + meta)
            return model

    # defining new reaction
    reaction = cobra.Reaction(react_name[0])
    reaction.name = 'SLIME reaction' + react_name[1]
    reaction.subsystem = 'SLIMEr reaction, Lipid metabolism'
    reaction.lower_bound = 0.  # This is the default
    reaction.upper_bound = 1000.  # This is the default 
    model.add_reactions([reaction])

    model.reactions.get_by_id(reaction.id).add_metabolites(meta_dict)
    print(model.reactions.get_by_id(reaction.id))

    return model


### Pseudo reaction head group

In [20]:
sid = 'pseudo_l_bkb'
sname = 'Pseudo lipid head backbone'
new_meta = cobra.Metabolite(id=sid,name=sname, compartment='cytoplasm')
ipfal21.add_metabolites([new_meta])

a2 = metas_df[metas_df.reaction=="2a"]
# change between different models
tester = dict(zip(metas_df['ids'],a2['mean_Ring ']*-1))
tester['pseudo_l_bkb'] = 1
create_reaction(ipfal21,['slimer_2a','pseudo head'], tester)
del tester

Reaction already exists, delete it first slimer_2a


In [21]:
tester

NameError: name 'tester' is not defined

In [50]:
sid = 'pseudo_sidchn'
sname = 'Pseudo side chain'
new_meta = cobra.Metabolite(id=sid,name=sname, compartment='cytoplasm')
ipfal21.add_metabolites([new_meta])

b2 = metas_df[metas_df.reaction=="2b"]
# change between different models
b2 = b2.dropna(subset=['mean_Ring '])
tester = dict(zip(b2['ids'],b2['mean_Ring ']*-1))
tester[sid] = 1
create_reaction(ipfal21,['slimer_2b','pseudo sid chain'], tester)

slimer_2b: 0.0799998972641441 c140_chain + 0.0695958371431213 c141_chain + 0.1088193595446747 c160_chain + 0.117059802541685 c161_chain + 0.1000785929297184 c180_chain + 0.2006046005116245 c181_chain + 0.0793690991092801 c200_chain + 0.1161146326679474 c201_chain + 0.000102735855841 c241_chain + 0.0002568396396026 c242_chain + 2.054717116820921e-05 c250_chain + 0.0044895569002537 c343_chain + 0.0184102653667156 c363_chain + 0.0267729640321768 c364_chain + 0.000102735855841 c387_chain + 0.0046333870984312 c406_chain + 0.0012944717835971 c407_chain + 0.01354058579985 c546_chain + 0.0053422645037344 c566_chain --> pseudo_sidchn


Name,iPfal19_v1
Memory address,7fe28e4811e0
Number of metabolites,1035
Number of reactions,1235
Number of genes,475
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"c, e, ap, m, fv"


In [49]:
ipfal21.reactions.remove('slimer_2b')

In [105]:
metas_df = pd.read_csv('Datasets/slimeR_PLs_MW.tsv',index_col=0)
metas_df = metas_df.drop(columns=metas_df.columns[-2:])
metas_df.index.map(SLIMEr_react_dict)

CE 16:0
CE 16:1
CE 18:1
CE 18:2
CE 18:3
CE 20:4
CE 20:5
DAG 16:0 16:0
DAG 16:0 18:0
DAG 16:0 18:1
DAG 18:0 18:1
DAG 18:1 18:1
DHSM 16:0
DHSM 17:0
DHSM 18:0
DHSM 19:0
DHSM 20:0
DHSM 22:0
DHSM 24:0
DHSM 25:0
PC 14:0 18:0
PC 16:0 16:0
PC 14:0 18:1
PC 14:1 18:0
PC 16:1 16:0
PC 14:0 20:0
PC 16:0 18:0
PC 14:0 20:1
PC 14:1 20:0
PC 16:0 18:1
PC 16:1 18:0
PC 14:1 20:1
PC 16:1 18:1
PC 16:0 20:0
PC 18:0 18:0
PC 16:0 20:1
PC 16:1 20:0
PC 18:1 18:0
PC 16:1 20:1
PC 18:1 18:1
PC 14:0 18:0
PC 16:0 16:0
PC 14:0 20:1
PC 14:1 20:0
PC 16:0 18:1
PC 16:1 18:0
PC 16:1 20:1
PC 18:1 18:1
PC 34:3
PC 36:3
PC 36:4
PC 36:5
PC 38:3
PC 38:4
PC 38:5
PC 38:6
PC 38:7
PC 40:4
PC 40:5
PC 40:6
PC 40:7
PC O-38:4
PC O-38:5
PE 14:0 20:1
PE 14:1 20:0
PE 16:0 18:1
PE 16:1 18:0
PE 14:1 20:1
PE 16:1 18:1
PE 16:0 20:1
PE 16:1 20:0
PE 18:1 18:0
PE 16:1 20:1
PE 18:1 18:1
PE 36:3
PE 36:4
PE 36:5
PE 38:3
PE 38:4
PE 38:5
PE 38:6
PE 40:5
PE 40:6
PG 16:0 18:1
PG 16:0 18:2
PG 18:0 18:1
PG 18:0 18:2
PG 18:1 18:1
PG 18:1 18:2
PS 14:0 20:1


Index([                         {'CE': 1, '16:0': 1, 'CE 16:0': -1},
                                {'CE': 1, '16:1': 1, 'CE 16:1': -1},
                                {'CE': 1, '18:1': 1, 'CE 18:1': -1},
                                {'CE': 1, '18:2': 1, 'CE 18:2': -1},
                                {'CE': 1, '18:3': 1, 'CE 18:3': -1},
                                {'CE': 1, '20:4': 1, 'CE 20:4': -1},
                                {'CE': 1, '20:5': 1, 'CE 20:5': -1},
                  {'DAG': 1, '16:0': 1, 'DAG (32:0) 16:0_16:0': -1},
       {'DAG': 1, '16:0': 1, '18:0': 1, 'DAG (34:0) 16:0_18:0': -1},
       {'DAG': 1, '16:0': 1, '18:1': 1, 'DAG (34:1) 16:0_18:1': -1},
       ...
             {'TAG': 1, '18:1': 1, 'TAG (54:3) 18:1_18:1_18:1': -1},
                              {'TAG': 1, '54:6': 1, 'TAG 54:6': -1},
                              {'TAG': 1, '56:6': 1, 'TAG 56:6': -1},
                   {'Cer': 1, '16:0': 1, '18:1': 1, 'Cer 16:0': -1},
                   {'Ce

In [106]:
metas_df

,mean_Ring,mean_Trophozoite,mean_Schizont,reaction,MW,Formula,Name,MetNetX,Model_id
CE 16:0,"-3,04666666666667","0,21","-2,86333333333333",p,"625,7",C43H76O2,Cholesterol-ester 16:0,MNXM730833,xolest160
CE 16:1,"-2,04","-1,59333333333333","-3,10333333333333",p,623,C43H74O2,Cholesterol-ester 16:1,MNXM736462,xolest161
CE 18:1,"-6,95333333333333","-5,32","-11,0166666666667",p,"651,1",C45H78O2,Cholesterol-ester 18:1,MNXM730830,xolest181
CE 18:2,"-24,6","-18,7933333333333","-36,3133333333333",p,"649,1",C45H76O2,Cholesterol-ester 18:2,MNXM730826,xolest182
CE 18:3,"-4,28333333333333","-2,84333333333333","-5,7",p,"647,1",C45H74O2,Cholesterol-ester 18:3,MNXM731672,xolest183
CE 20:4,"-3,54666666666667","-2,60333333333333","-5,03333333333333",p,"673,1",C47H76O2,Cholesterol-ester 20:4,MNXM1104132,xolest204
CE 20:5,"-0,403333333333333","-0,403333333333333","-0,403333333333333",p,"671,1",C47H74O2,Cholesterol-ester 20:5,MNXM45917,xolest205
DAG (32:0) 16:0_16:0,0,"27,7955555555556","37,1466666666667",p,"568,9",C35H68O5,Diacylglycerol (32:0) 16:0_16:0,MNXM3132,12dgr160
DAG (34:0) 16:0_18:0,0,"10,6233333333333","11,19",p,"596,537975",C37H72O5,Diacylglycerol (34:0) 16:0_18:0,MNXM49379,12dgr160180
DAG (34:1) 16:0_18:1,0,"25,4666666666667","124,12",p,"594,9",C37H70O5,Diacylglycerol (34:1) 16:0_18:1,MNXM49574,12dgr1601819Z
